In [1]:
import sys
sys.path.append("/Users/anuram/Documents/Documents/hydrogenase-ftir/src")
sys.path.append("/Users/anuram/Documents/Documents/hydrogenase-ftir/notebooks/anu_winter_2024")
#sys.path.append("/Users/anuram/Documents/Documents/hydrogenase-ftir/src/hydrogenase_processing")

In [4]:
%load_ext autoreload
%autoreload 2

#Setting Up and Importing the Necessary Packages/Libraries
##Package for reading in Bruker OPUS type files
from brukeropusreader import read_file
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
import pathlib

#Local Functions
from hydrogenase_processing.cut_range import cut_range_subtract_multiple_wv
from hydrogenase_processing.second_deriv import  second_deriv_batch
from hydrogenase_processing.anchor_points import interact
from hydrogenase_processing.anchor_points import get_peaks_absorbance, get_peak_wid_at_half_height
from hydrogenase_processing.baseline import baseline_spline, baseline_correction, get_baseline_peak_index

#from hydrogenase_processing.anchor_points import get_peaks, get_start_end_anchorpoints, get_all_anchor_points, baseline_spline, get_peaks_absorbance, baseline_correction, get_peak_baseline_absorbance, plot_baseline_data
from hydrogenase_processing.peak_fit import peak_fit
from hydrogenase_processing.io import import_run_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Importing spectra data and water vapor data

In [5]:
#Paths to Local Data
path_to_water_vapor_data = pathlib.Path("../../data/opus_files/water_vapor")
path_to_output_plots_= pathlib.Path("../../data/output_plots/")
path_to_pD6_data = pathlib.Path("../../data/opus_files/pD6")

In [6]:
pD6_raw_data = import_run_data(path_to_pD6_data, input_type="raw spectra", output_folder = '/Users/anuram/Documents/output_ftir')
wv_data = import_run_data(path_to_water_vapor_data)

In [10]:
pD6_raw_data[1].raw_data

{'Text Information': '4\x00\x00x\x00\x00\x00\x18\x00\x00\x00`\x00\x00\x00J\x00\x00\x00ø\x01\x00\x00@\x00\x00\x00\x1e\x00\x00\x00 \x03\x00\x000\x00\x00\x006\x00\x00\x00\x98\x03\x00\x00\x07\x04\x00\x00ä\x06\x00\x00\x90\x05\x00\x00\x0f\x10\x00\x00ä\x06\x00\x00\x90!\x00\x00(\x00\x00\x00\x98\x00\x00\x00\x90=\x00\x00h\x00\x00\x00\\\x00\x00\x00ð?\x00\x008\x00\x00\x00T\x00\x00\x00`A\x00\x00H\x00\x00\x00\x1e\x00\x00\x00°B\x00\x00\x1b\x04\x00\x00,\x00\x00\x00(C\x00\x00\x0b\x04\x00\x00ê\x06\x00\x00ØC\x00\x00\x00\x00h@\x92\x02\x00\x00\x80_\x00\x00\x1f\x10\x00\x00,\x00\x00\x00Èi\x00\x00\x17\x04\x00\x00,\x00\x00\x00xj\x00\x00\xa0\x00\x00\x00T\x00\x00\x00(k\x00\x00 \x00\x00\x00\x96\x00\x00\x00xl',
 'Optik': {'ACC': 'Hyperion 1000/2000 Transmission 15x Objective #HYPERI01_05',
  'APR': 0,
  'APT': '6 mm',
  'BMS': 'KBr',
  'CHN': 'Left Exit',
  'DTC': 'Hyperion MCT Left channel [Microscope]',
  'HPF': '0',
  'LPF': '10.',
  'LPV': 4000,
  'OPF': 'Open',
  'PGN': '0',
  'RDX': '0',
  'SRC': 'MIR',
  'V

In [ ]:
from hydrogenase_processing.cut_range import cut_range_subtraction_multiple_wv
cut_spectra = cut_range_subtraction_multiple_wv(pD6_raw_data[0].get_raw_data(), wv_data)
print(cut_spectra)

In [11]:
pD6_raw_data[0].get_raw_data()

{'Text Information': '4\x00\x00x\x00\x00\x00\x18\x00\x00\x00`\x00\x00\x00J\x00\x00\x00ø\x01\x00\x00@\x00\x00\x00\x1e\x00\x00\x00 \x03\x00\x000\x00\x00\x006\x00\x00\x00\x98\x03\x00\x00\x07\x04\x00\x00ä\x06\x00\x00\x90\x05\x00\x00\x0f\x10\x00\x00ä\x06\x00\x00\x90!\x00\x00(\x00\x00\x00\x98\x00\x00\x00\x90=\x00\x00h\x00\x00\x00\\\x00\x00\x00ð?\x00\x008\x00\x00\x00T\x00\x00\x00`A\x00\x00H\x00\x00\x00\x1e\x00\x00\x00°B\x00\x00\x1b\x04\x00\x00,\x00\x00\x00(C\x00\x00\x0b\x04\x00\x00ê\x06\x00\x00ØC\x00\x00\x00\x00h@\x92\x02\x00\x00\x80_\x00\x00\x1f\x10\x00\x00,\x00\x00\x00Èi\x00\x00\x17\x04\x00\x00,\x00\x00\x00xj\x00\x00\xa0\x00\x00\x00T\x00\x00\x00(k\x00\x00 \x00\x00\x00\x96\x00\x00\x00xl',
 'Optik': {'ACC': 'Hyperion 1000/2000 Transmission 15x Objective #HYPERI01_05',
  'APR': 0,
  'APT': '6 mm',
  'BMS': 'KBr',
  'CHN': 'Left Exit',
  'DTC': 'Hyperion MCT Left channel [Microscope]',
  'HPF': '0',
  'LPF': '10.',
  'LPV': 4000,
  'OPF': 'Open',
  'PGN': '0',
  'RDX': '0',
  'SRC': 'MIR',
  'V

In [12]:
for index,prospecpy_obj in enumerate(pD6_raw_data):
    print(index, prospecpy_obj.get_raw_data())

0 {'Text Information': '4\x00\x00x\x00\x00\x00\x18\x00\x00\x00`\x00\x00\x00J\x00\x00\x00ø\x01\x00\x00@\x00\x00\x00\x1e\x00\x00\x00 \x03\x00\x000\x00\x00\x006\x00\x00\x00\x98\x03\x00\x00\x07\x04\x00\x00ä\x06\x00\x00\x90\x05\x00\x00\x0f\x10\x00\x00ä\x06\x00\x00\x90!\x00\x00(\x00\x00\x00\x98\x00\x00\x00\x90=\x00\x00h\x00\x00\x00\\\x00\x00\x00ð?\x00\x008\x00\x00\x00T\x00\x00\x00`A\x00\x00H\x00\x00\x00\x1e\x00\x00\x00°B\x00\x00\x1b\x04\x00\x00,\x00\x00\x00(C\x00\x00\x0b\x04\x00\x00ê\x06\x00\x00ØC\x00\x00\x00\x00h@\x92\x02\x00\x00\x80_\x00\x00\x1f\x10\x00\x00,\x00\x00\x00Èi\x00\x00\x17\x04\x00\x00,\x00\x00\x00xj\x00\x00\xa0\x00\x00\x00T\x00\x00\x00(k\x00\x00 \x00\x00\x00\x96\x00\x00\x00xl', 'Optik': {'ACC': 'Hyperion 1000/2000 Transmission 15x Objective #HYPERI01_05', 'APR': 0, 'APT': '6 mm', 'BMS': 'KBr', 'CHN': 'Left Exit', 'DTC': 'Hyperion MCT Left channel [Microscope]', 'HPF': '0', 'LPF': '10.', 'LPV': 4000, 'OPF': 'Open', 'PGN': '0', 'RDX': '0', 'SRC': 'MIR', 'VEL': '20.', 'SON': 'Off'}

In [13]:
pD6_raw_data[0].cut_range_subtract_single_wv(wv_data)

IndexError: index 0 is out of bounds for axis 0 with size 0

# Subtracting water vapor from raw spectra

In [19]:
#Initializing dict of post water vapor subtraction spectra
cut_range_sub_wv_data = cut_range_subtract_multiple_wv(pD6_raw_data, wv_data, range_start = 2150, range_end = 1850)
#pD6_raw_data[].cut_subtracted_data

959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115
959 1115


In [20]:
pD6_raw_data

In [ ]:
pD6_raw_data[0].cut_range_subtract_single_wv

## Plot the subtracted data for visual inspection

In [ ]:
sample_raw = cut_range_sub_wv_data["Hyd1 pD6 ACT.0008"]
sample_raw[0][0].plot()

print(sample_raw[0][0].fit_atm_params)

## Get the second derivatives

In [ ]:
second_deriv_pD6_data = second_deriv_batch(cut_range_sub_wv_data)

# Baseline correction and cruve fit for an example spectra workflow:

In [ ]:
sample_second_deriv = second_deriv_pD6_data["Hyd1 pD6 ACT.0005_second_deriv"]
#print(sample_second_deriv)

## Using a user interface please select the appropriate number of peaks using the threshold variable and number of anchor points using adjustment factor variable

In [ ]:
#anchor_point_dict, deriv_x_peak_val, anchor_points_raw_data, y_corr_abs
anchor_point_dict, deriv_x_peak_val, anchor_points_raw_data, y_corr_abs = interact(sample_second_deriv, sample_raw, 0.4, 1.8)

## Fit baseline spline through these anchor points to be used for baseline correction of raw spectra

In [ ]:
baseline_curve = baseline_spline(anchor_point_dict, smooth=10**-9)
peak_wv, peak_abs = get_peaks_absorbance(deriv_x_peak_val, anchor_points_raw_data, y_corr_abs)

plt.plot(anchor_points_raw_data, y_corr_abs, label = 'raw data')
plt.plot(baseline_curve['wavenumber'], baseline_curve['absorbance'], 'r-', label = 'baseline spline')
plt.plot(peak_wv, peak_abs, 'go', label = 'peaks')
plt.ylabel('Absorbance')
plt.xlabel('wavenumber')
plt.legend()

## Subtract the baseline spline from the raw data and plot the results

In [ ]:
baseline_corrected_abs = baseline_correction(baseline_curve, anchor_points_raw_data, y_corr_abs)
peak_wv_index, peak_wv_baseline, peak_baseline_abs = get_baseline_peak_index(baseline_corrected_abs, anchor_points_raw_data, peak_wv)


# Get peak widths at half peak height

In [ ]:
peakwidth_at_half_height = get_peak_wid_at_half_height(baseline_corrected_abs, peak_wv_index)
print(peakwidth_at_half_height)

# Fitting gaussian and lorentizian curves on the baseline corrected data

In [ ]:
params, rmse = peak_fit('Gaussian', anchor_points_raw_data, baseline_corrected_abs, peak_wv_index)
print(f'Root mean squared error is {rmse}')

In [ ]:
params, rmse = peak_fit('Lorentzian', anchor_points_raw_data, baseline_corrected_abs, peak_wv_index)
print(f'Root mean squared error is {rmse}')